#  view를 개선하기

> 모든 페이지에 항상 나오는 내용
- 저작권, 주소 등...반복되는 요소들을 매번 html마다 반복하지 않고, 한번에 처리

> base template, sub template
- 템플릿 안에 템플릿을 넣는 개념
- 변경되는 템블릿만 수정
- base template: 저작권, 기본사항 등 -> 한번만 하기
- sub template: 안의 부분만 rendering하는 부분

# 1. base templage만들기: base.html

> base template를 사용하려면
- sub template에는 base를 import하고, base 안어 들어갈 부분을 block으로 묶기
- base teplate에서는 sub template를 넣을 부분을 block문으로 알려주기

In [ ]:
# base.html
'''
My blog<br>


<!--  나중에 바뀌는 부분-> 페이지에서 rendering되는 부분 -->
{% block content %}
{% endblock %}


copy right.....<br>
서울 특별시

'''

In [ ]:
# list.html
'''
<!-- import, includ 개념 -->
{% extends 'blog/base.html' %}


<!-- base template안의 위치 지정 -->

{% block content %}

sub-template<br>

{% for d in data%}
    {{d.title}}<br>
    {{d.text|linebreaks}}  
    <!-- linebreaks: enter -> <br>로 바꿔줌 -->
    <hr>
{%endfor%}

{% endblock %}

'''

# 2. detail view만들기: title을 클릭하면 게시물 1개를 보여주기

#### 동적 url로 title 보여주기

In [ ]:
# liist.htmp
'''
<!-- import, includ 개념 -->
{% extends 'blog/base.html' %}


<!-- base template안의 위치 지정 -->

{% block content %}

sub-template<br>

{% for d in data%}

<!-- get 방식으로 보내려면: a href: /blog/id =... -->
<!-- 동적 url을 가져오려면 -->
    <a href="{{d.pk}}/detail"></a>{{d.title}}</a><br>
    <!-- {{d.text|linebreaks}}    # 목록만 보여주도록 하기 위해서 빼기-->
    <!-- linebreaks: enter -> <br>로 바꿔줌 -->
    <hr>
{%endfor%}

{% endblock %}


'''

In [ ]:
# detail.html
'''

<!-- import, includ 개념 -->
{% extends 'blog/base.html' %}


<!-- base template안의 위치 지정 -->

{% block content %}

게시물을 보여줘<br>

    {{d.text|linebreaks}}  
    <!-- linebreaks: enter -> <br>로 바꿔줌 -->


{% endblock %}

'''

In [ ]:
# view.py
'''
def list(request):
    data =Post.objects.all()
    return render (request,'blog/list.html', {'data':data})

def detail(request, pk):
    p = get_object_or_404(Post, pk = pk)
    return render(request, 'blog/detail.html', {'d': p})
'''

# 3. View를 객체로 바꾸기
- 특정 url이 들어오면 함수를 호출하는 개념
- 모델 만들것처럼 view 객체를 만들어서 처리


> 항상 url이 2개 (login, loginform)
- request.method == post, get으로 if문으로 분기할 수도 있고
- class를 사용할 수도 있고

## 3.1. 연습

In [ ]:
# urls.py

'''
urlpatterns = [
    path('', views.index),
    # 동적 url 다루기
    path('<name>/', views.index2),  # 임의의 문자가 다 올 수 있음.
    # path('<int:pk>/detail', views.index3),
    path('<int:pk>/detail', views.detail),


    # 함수 기반
    path('list', views.list)

    # class 기반
    path('list2', views.PostView.as_view())    # as_view()함수를 가지고 PostView 객체 생성
]

'''

In [ ]:
# views.py
'''
# View를 상속받아서 정의
# get, post 나눠서 정의
# 특정 url이 호출되면 PoseView instance가 생성
class PostView(View):
        
    def get(self, request):
        return HttpResponse('get ok~~')

    def post(self, request):
        return HttpResponse('post ok~~~')

'''

> 실행
- 127.0.0.1:8000/blog/list2
- get 방식으로 접속
- post로 하려면 form tag로 사용해야 함.!!

## 3.2. login을 class로 처리하기

In [ ]:
# urls.py
'''
urlpatterns = [
    path('', views.index),
    # 동적 url 다루기
    path('<name>/', views.index2),  # 임의의 문자가 다 올 수 있음.
    # path('<int:pk>/detail', views.index3),
    path('<int:pk>/detail', views.detail),


    # 함수 기반
    path('list', views.list),

    # class 기반
    path('list2', views.PostView.as_view()),    # as_view()함수를 가지고 PostView 객체 생성
    path('login', views.LoginView.as_view()),
]

'''

In [ ]:
# login.html
'''
<form action = '' method = post>
    {% csrf_token %}
    username<input type=text name = username> <br>
    password <input type = password name = password><br>
    <input type = submit value = 'login'>
</form>

'''

In [ ]:
# views.py
'''
class LoginView(View):
    def get(self, request):
        return render(request, 'blog/login.html')



    def post(self, request):
        
        return redirect('list')

'''

> 실행
- http://127.0.0.1:8000/blog/login

### pass 암호하는 방법
- 장고가 사용하는 인증방법 사용해야 함.
- home pass: home

In [ ]:
'''
from django.contrib.auth import authenticate


user = authenticate(usernema = 'home', password = '1234')
user
''''''


In [ ]:
> views.py
'''
class LoginView(View):
    def get(self, request):
        return render(request, 'blog/login.html')

    def post(self, request):

        # user table의 password는 암호화되어 들어가있음.-> 간접적으로 비교 해야 함.
        username = request.POST.get('username')
        password = request.POST.get('password')

        user = authenticate(username =username, password = password)
        if user == None:

            # return HttpResponse('암호 틀림')
            return redirect('/blog/login')
        # return HttpResponse('인증 완료')
        return redirect('/blog/list')


'''



## 3.3. 경로를 name으로 처리하기
- redirect('name')쓰기

In [ ]:
# urls.py
'''
urlpatterns = [
    path('', views.index),
    # 동적 url 다루기
    path('<name>/', views.index2),  # 임의의 문자가 다 올 수 있음.
    # path('<int:pk>/detail', views.index3),
    path('<int:pk>/detail', views.detail),


    # 함수 기반
    path('list', views.list, name = 'list'),

    # class 기반
    path('list2', views.PostView.as_view()),    # as_view()함수를 가지고 PostView 객체 생성
    path('login', views.LoginView.as_view()), name = 'login',

    # 이름: 경로 신경쓰지 말고 name을 경로로 사용할 수 있음.
]
'''

In [ ]:
#views.py
'''
class LoginView(View):
    def get(self, request):
        return render(request, 'blog/login.html')

    def post(self, request):

        # user table의 password는 암호화되어 들어가있음.-> 간접적으로 비교 해야 함.
        username = request.POST.get('username')
        password = request.POST.get('password')

        # 장고에서 지원하는 함수
        user = authenticate(username =username, password = password)
        if user == None:

            # return HttpResponse('암호 틀림')
            # return redirect('/blog/login')
            return redirect('login')
        # return HttpResponse('인증 완료')
        # return redirect('/blog/list'): web 표준은 경로명(/를 써서 urls.py에서 찾음)을 쓰는 것이 맞음.
        return redirect('list')

'''